# AI Manga Upscale Colab

<br>Upscale black-and-white and color manga pages by 4x using AI.
<br><br>You may want to use the post-production toolset (downscaler, page renamer, cbz creator) - [AI Manga Upscale Colab Post-Production Toolset](https://colab.research.google.com/github/Spladenly/ai-manga-upscale-colab/blob/main/post-prod.ipynb).<br><br>

> When running the script, you will need to grant access to your Google Drive. This permission will be used exclusively to access the folder: "/simple_upscaler". No other personal folders or files will be used.

<br>

Folder structure on Google Drive:

/simple_upscaler/bw (black-and-white manga pages go here) *
<br>/simple_upscaler/color (color manga pages go here) *
<br>/simple_upscaler/output (upscaled manga pages will appear here)
<br><br>Manga pages can also be uploaded to subfolders.
<br><br> * Supported files: bmp, dib, jpeg, jpg, jpe, jp2, png, webp, pbm, pgm, ppm, pxm, pnm, pfm, sr, ras, tiff, tif, exr, hdr, pic, gif, tga.<br><br>

> Google Drive has a delay. If you've just uploaded or deleted files -- Google Colab may not see the changes for up to 1 minute. It works the same way in the other direction: if processing has just finished -- you won't see new files on Google Drive right away. Please wait.

<br>To begin processing -- upload manga pages to previously mentioned folder(s), and click the buttons by order:<br><br>
>"Setup Upscaler"<br><br>
>"Run Manga Upscaler"

In [ ]:
# @title Setup Upscaler
import os

if not os.path.exists("simple_upscaler"):
  print("Downloading upscaler along with the AI models...")
  !git clone https://github.com/thefirst632student/simple_upscaler.git
if not os.path.exists("/content/simple_upscaler/models/4x_MangaJaNai_1200p_V1_ESRGAN_70k.pth"):
  !wget https://github.com/the-database/MangaJaNai/releases/download/1.0.0/MangaJaNai_V1_ModelsOnly.zip -O /content/MangaJaNai_V1_ModelsOnly.zip
  !unzip /content/MangaJaNai_V1_ModelsOnly.zip -d /content/simple_upscaler/models/
os.environ['bw_selection'] = '/content/simple_upscaler/models/4x_MangaJaNai_1200p_V1_ESRGAN_70k.pth'
os.environ['color_selection'] = '/content/simple_upscaler/models/4x_IllustrationJaNai_V2standard_DAT2_27k.safetensors'
print("Setup completed!")

In [ ]:
# @title Select black white upscale model
import ipywidgets as widgets
import glob
import os

# Get all model files in the simple_upscaler directory
list_files = glob.glob('/content/simple_upscaler/models/*')
all_models = sorted(list_files)

# Filter files containing "MangaJaNai" in their names
bw_models = sorted([f for f in all_models if "mangajanai" in f.lower()])

# Create a checkbox widget
bw_checkbox = widgets.Checkbox(
    value=False,
    description='Show all files',
    disabled=False,
    indent=False
)

# Create a dropdown widget
bw_dropdown = widgets.Dropdown(
    options=bw_models,
    description='Select a model file:',
    disabled=False,
)

# Display the checkbox and dropdown
display(bw_checkbox)
display(bw_dropdown)

# Function to update the dropdown options based on the checkbox state
def update_bw_dropdown_options(change):
    current_value = bw_dropdown.value # Store the current value
    if bw_checkbox.value:
        bw_dropdown.options = all_models
    else:
        bw_dropdown.options = bw_models

    # Attempt to restore the previously selected value if it exists in the new options
    if current_value in bw_dropdown.options:
        bw_dropdown.value = current_value
    else:
        # If the previous value is not in the new options, set to the first option
        if bw_dropdown.options: # Check if the options list is not empty
             bw_dropdown.value = bw_dropdown.options[0]

# Function to update the bw_selection environment variable and print it
def bw_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        global bw_selection
        bw_selection = change['new']
        os.environ['bw_selection'] = bw_selection # Set environment variable
        print(f"Selected file: {bw_selection}")

bw_checkbox.observe(update_bw_dropdown_options, names='value')
bw_dropdown.observe(bw_on_change)

In [ ]:
# @title Select color upscale model
import ipywidgets as widgets
import glob
import os

# Get all model files in the simple_upscaler directory
list_files = glob.glob('/content/simple_upscaler/models/*')
all_models = sorted(list_files)

# Filter files containing "IllustrationJaNai" in their names
color_models = sorted([f for f in all_models if "illustrationjanai" in f.lower()])

# Create a checkbox widget
color_checkbox = widgets.Checkbox(
    value=False,
    description='Show all files',
    disabled=False,
    indent=False
)

# Create a dropdown widget
color_dropdown = widgets.Dropdown(
    options=color_models,
    description='Select a model file:',
    disabled=False,
)

# Display the checkbox and dropdown
display(color_checkbox)
display(color_dropdown)

# Function to update the dropdown options based on the checkbox state
def update_color_dropdown_options(change):
    current_value = color_dropdown.value # Store the current value
    if color_checkbox.value:
        color_dropdown.options = all_models
    else:
        color_dropdown.options = color_models

    # Attempt to restore the previously selected value if it exists in the new options
    if current_value in color_dropdown.options:
        color_dropdown.value = current_value
    else:
        # If the previous value is not in the new options, set to the first option
        if color_dropdown.options: # Check if the options list is not empty
             color_dropdown.value = color_dropdown.options[0]

# Function to update the color_selection environment variable and print it
def color_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        global color_selection
        color_selection = change['new']
        os.environ['color_selection'] = color_selection # Set environment variable
        print(f"Selected file: {color_selection}")

color_checkbox.observe(update_color_dropdown_options, names='value')
color_dropdown.observe(color_on_change)

In [ ]:
# @title Run Manga Upscaler { display-mode: "form" }

import os
from google.colab import drive
from pathlib import Path
import torch

def check_connect_gdrive():
  if not os.path.exists("/content/gdrive/MyDrive"):
    print("Google Drive connection in progress...")
    drive.mount("/content/gdrive")

def init_dirs():
  Path("/content/gdrive/MyDrive/simple_upscaler").mkdir \
   (parents=True, exist_ok=True)
  Path("/content/gdrive/MyDrive/simple_upscaler/bw").mkdir \
   (parents=True, exist_ok=True)
  Path("/content/gdrive/MyDrive/simple_upscaler/color").mkdir \
   (parents=True, exist_ok=True)
  Path("/content/gdrive/MyDrive/simple_upscaler/output").mkdir \
   (parents=True, exist_ok=True)

def dir_contains_files(path):
  for root, dirs, files in os.walk(path):
    if files:
      return True
  return False

def ai_process_bw():
  !python simple_upscaler/upscale.py -se -i /content/gdrive/MyDrive/simple_upscaler/bw \
  -o /content/gdrive/MyDrive/simple_upscaler/output \
  $bw_selection

def ai_process_color():
  !python simple_upscaler/upscale.py -se -i /content/gdrive/MyDrive/simple_upscaler/color \
  -o /content/gdrive/MyDrive/simple_upscaler/output \
  $color_selection

def main():
  print("[AI Manga Upscale Colab] Manga Upscaler")

  if not torch.cuda.is_available():
    print("This session doesn't have a GPU.\n To connect a GPU, click:\n'Edit' -> 'Notebook settings' -> 'Hardware accelerator' = GPU; 'GPU type' = T4.\nAfter that, run this script again.")
    return

  check_connect_gdrive()
  init_dirs()

  status = 0

  if dir_contains_files("/content/gdrive/MyDrive/simple_upscaler/bw"):
    status += 1
    print("Upscaling bw...")
    ai_process_bw()

  if dir_contains_files("/content/gdrive/MyDrive/simple_upscaler/color"):
    status += 1
    print("Upscaling color...")
    ai_process_color()

  if status == 0:
    print("No pages were found in the following directories: '/simple_upscaler/bw' and '/simple_upscaler/color' on your Google Drive.\nPlease upload manga pages there, and run this script again.")
  else:
    print("The processing has been finished. The result can be downloaded from '/simple_upscaler/output' on your Google Drive.\nTo process additional pages, run this script again. If you don't plan to process additional pages in the near future, please close the current session.\nThis can be done by clicking on: the inverted triangle (next to the 'RAM' and 'Disk' labels in the upper right corner) -> 'Disable and remove runtime'.\nThis will unlock the resources reserved for this session for other users.")

if __name__ == "__main__":
  main()

You may also upload CBZs and ZIPs instead of images. Use the following script to extract them before running Upscale Manga. After the extraction, the original files will be moved to recycle bin.

In [ ]:
# @title Extractor { display-mode: "form" }

import os
import shutil
import zipfile
from google.colab import drive

def check_connect_gdrive():
  if not os.path.exists("/content/gdrive/MyDrive"):
    print("Google Drive connection in progress...")
    drive.mount("/content/gdrive")

def unpack(path):
  zip_files = [f for f in os.listdir(path) if f.endswith(".zip") or \
               f.endswith(".cbz")]

  print(f"Found {len(zip_files)} file(s).")

  for zip_file in zip_files:
      print(f"Extracting {zip_file}...")

      zip_path = os.path.join(path, zip_file)

      if not os.path.exists(os.path.splitext(zip_path)[0]):
        extract_folder = os.path.join(path, \
                                        os.path.splitext(zip_file)[0])
        os.makedirs(extract_folder, exist_ok=True)

        with zipfile.ZipFile(zip_path, "r") as zf:
            zf.extractall(extract_folder)

        os.remove(zip_path)
      else:
         print("The folder already exists -- skipping.")

def main():
  print("[AI Manga Upscale Colab] Extractor")

  check_connect_gdrive()

  status = 0
  bw_path = "/content/gdrive/MyDrive/ESRGAN/bw"
  color_path = "/content/gdrive/MyDrive/ESRGAN/color"

  if os.path.exists(bw_path):
    status += 1
    print(f"Scanning: {bw_path}")
    unpack(bw_path)

  if os.path.exists(color_path):
    status += 1
    print(f"Scanning: {color_path}")
    unpack(color_path)

  if status == 0:
    print("Both 'bw' and 'color' directories don't exist.")

  print("Done.")

if __name__ == "__main__":
  main()